- 方法相同、拟合参数表达不同 ：  模拟拟合参数不同，EQ13
- 单一方程式 ：预测与excel有差异 ，excel预测值是“联立方程的预测结果”？
- 单一方程参数 = 联立方程参数 ？ 基线拟合参数 = 场景拟合参数

In [1]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import time

from scipy.stats import kstest
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
dir = r"./data/"
print(os.listdir(dir))  #手术种类

['【20200703】-预测误差(2015-2017).xls', '【20200703】-模拟（2016，10）-3.xls', '【20200703】-模拟（2016，6-8）-2.xls', '【20200703】-模拟（2016,1-4）-1.xls', '其它数据.csv', '.ipynb_checkpoints']


# 数据源

## excel数据源
- 通过"预测误差表"获得数据源；

- 也可以通过模拟误差获得数据源，但是那样更复杂一些。

In [3]:
file = "【20200703】-预测误差(2015-2017).xls"
df_e1= pd.read_excel(dir+file,"e1",index_col = 0 ,na_values=['9999'])
df_e1 = df_e1.iloc[:list(df_e1.index.isna()).index(True),:]  #取index为na前所有行的数据
print(df_e1.index)

Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020],
      dtype='object', name='obs')


In [4]:
df_e2= pd.read_excel(dir+file,"e2",index_col = 0 ,na_values=['9999'])
df_e2 = df_e2.iloc[:list(df_e2.index.isna()).index(True),:]
print(df_e2.index)

Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020],
      dtype='object', name='obs')


In [5]:
df_e3= pd.read_excel(dir+file,"e3",index_col = 0 ,na_values=['9999'])
df_e3 = df_e3.iloc[:list(df_e3.index.isna()).index(True),:]
print(df_e3.index)

Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020],
      dtype='object', name='obs')


In [6]:
df_es = df_e1.merge(df_e2,how = "left" ,left_index =True ,right_index =True)
df_es = df_es.merge(df_e3,how = "left" ,left_index =True ,right_index =True)
df_es.columns = df_es.columns.str.lower()
list(df_es.columns )

['con_fn_bj',
 'con_fn_bj_0',
 'con_fn_hb',
 'con_fn_hb_0',
 'con_fn_tj',
 'con_fn_tj_0',
 'con_fr_bj',
 'con_fr_bj_0',
 'con_fr_tj',
 'con_fr_tj_0',
 'con_gn_bj',
 'con_gn_bj_0',
 'con_gn_tj',
 'con_gn_tj_0',
 'con_gr_bj',
 'con_gr_bj_0',
 'con_gr_tj',
 'con_gr_tj_0',
 'con_pn_bj',
 'con_pn_bj_0',
 'con_pn_tj',
 'con_pn_tj_0',
 'con_pr_bj',
 'con_pr_bj_0',
 'con_pr_tj',
 'con_pr_tj_0',
 'con_prn_bj',
 'con_prn_bj_0',
 'con_prn_tj',
 'con_prn_tj_0',
 'con_prr_bj',
 'con_prr_bj_0',
 'con_prr_tj',
 'con_prr_tj_0',
 'con_pun_bj',
 'con_pun_bj_0',
 'con_pun_tj',
 'con_pun_tj_0',
 'con_pur_bj',
 'con_pur_bj_0',
 'con_pur_tj',
 'con_pur_tj_0',
 'cone_rn_bj',
 'cone_rn_bj_0',
 'cone_rn_hb',
 'cone_rn_hb_0',
 'cone_rn_tj',
 'cone_rn_tj_0',
 'cone_un_bj',
 'cone_un_bj_0',
 'cone_un_hb',
 'cone_un_hb_0',
 'cone_un_tj',
 'cone_un_tj_0',
 'conet_rn_bj',
 'conet_rn_bj_0',
 'conet_rn_hb',
 'conet_rn_hb_0',
 'conet_rn_tj',
 'conet_rn_tj_0',
 'conet_un_bj',
 'conet_un_bj_0',
 'conet_un_hb',
 'conet_un

## Eviews数据源
一定要右键黏贴

In [25]:
df_other = pd.read_csv("./data/其它数据.csv",index_col=0)
df_other

,d_gdp1r_bj,cul_area_bj,d_gdp2r_bj_1,exrate_cn,d_gdp2_indar_bj,pcon_gd_bj,pcon_pud_bj,pcon_prd_bj,pinv_fon_bj,inv_chr_bj,pinv_chd_bj,consn_bj,inv_fir_bj,pinv_fid_bj,con_fn_bj,con_fr_bj,nex_gsr_bj,pnex_gsd_bj
1996,0.0,34.400000,0.0,831.42,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1061.6000,NaN,NaN,1010.0,NaN,NaN,NaN
1997,0.0,34.200000,0.0,828.98,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1208.5000,NaN,NaN,1199.1,NaN,NaN,NaN
1998,0.0,34.100000,0.0,827.91,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1373.6000,NaN,NaN,1307.1,NaN,NaN,NaN
1999,0.0,33.800000,0.0,827.83,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1509.3000,NaN,NaN,1482.1,NaN,NaN,NaN
2000,0.0,32.900000,0.0,827.84,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1658.7000,NaN,NaN,1636.0,NaN,NaN,NaN
2001,0.2,29.200000,0.0,827.70,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1831.4000,NaN,NaN,1859.1,NaN,NaN,NaN
2002,0.1,27.500000,0.0,827.70,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2005.2000,NaN,NaN,2232.0,NaN,NaN,NaN
2003,-0.3,26.000000,0.0,827.70,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2296.9000,NaN,NaN,2558.0,NaN,NaN,NaN
2004,0.0,23.600000,0.0,827.68,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2626.6000,NaN,NaN,3002.7,NaN,NaN,NaN
2005,-0.5,23.300000,-0.5,819.17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2911.7000,NaN,NaN,3401.8,NaN,NaN,NaN


## 衍生变量
- 根据定义式衍生的变量

In [8]:
df_der = pd.DataFrame()
df_der["gdp2_indad_bj"] = df_es["gdp2_indan_bj"]/df_es["gdp2_indar_bj"]*100
df_der["gdp2_bindad_bj"] = df_es["gdp2_bindan_bj"]/df_es["gdp2_bindar_bj"]*100
df_der["gdp2r_erro_bj"] = df_es["gdp2r_bj"]-df_es["gdp2_indar_bj"]-df_es["gdp2_bindar_bj"]
df_der["gdplr_bj_s1"] = df_es["gdp1r_bj"].shift(1)   #直接写进表达式，结果有差异
df_der["gdp2r_bj1_s1"] = df_es["gdp2r_bj1"].shift(1)  #
df_der

,gdp2_indad_bj,gdp2_bindad_bj,gdp2r_erro_bj,gdplr_bj_s1,gdp2r_bj1_s1
obs,,,,,
1996,83.709520,74.150505,18.010553,NaN,NaN
1997,85.047474,74.365462,9.446457,124.041239,897.383655
1998,82.595662,75.936530,5.434022,126.272872,958.614649
1999,79.152690,75.781070,10.995119,127.361097,1049.066999
2000,81.639835,76.727743,11.335207,130.820954,1177.348150
2001,82.435618,77.414788,7.835879,134.439627,1305.811813
2002,83.107575,78.441288,4.143980,138.971821,1422.533804
2003,88.825732,81.372863,6.037147,142.686523,1541.245248
2004,94.522739,87.173392,11.836135,140.573841,1725.142316


# 定义
## 参数
- 时间段 ： 1996-2008 ,每个表达式的时间段可能不一样
- 数据源 ： [df_es,df_other,df_der]

In [29]:
df_list = [df_es,df_other,df_der]

## 函数

In [30]:
def whereis (list1,df_list = df_list ) :
    for col1 in list1 :
        for i in range(len(df_list )):
            df = df_list[i]
            if  col1 in list(df.columns):
                print(col1 , "在第 ",i+1," 个数据框中")

# 北京
## Eq 1 : gdp1r_bj 行为方程 ok
行为方程 ： gdp1r_bj  = cul_area_bj + gdp1r_bj(-1) + d_gdp1r_bj

In [111]:
test1 = pd.concat([df_es["gdp1r_bj"],df_other[["cul_area_bj","d_gdp1r_bj"]],df_der["gdplr_bj_s1"]],axis =1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1.head(2)

,gdp1r_bj,cul_area_bj,d_gdp1r_bj,gdplr_bj_s1
obs,,,,
1997,126.272872,34.2,0.0,124.041239
1998,127.361097,34.1,0.0,126.272872


In [112]:
results = sm.ols(formula = "gdp1r_bj ~ cul_area_bj  + gdplr_bj_s1 + d_gdp1r_bj",data = test1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               gdp1r_bj   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     552.1
Date:                Wed, 08 Jul 2020   Prob (F-statistic):           3.91e-17
Time:                        10:07:52   Log-Likelihood:                -29.414
No. Observations:                  21   AIC:                             66.83
Df Residuals:                      17   BIC:                             71.01
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.0219      6.900     -0.148      0.884     -15.580      13.536
cul_area_bj     0.1348      0.080      1.675      0.112      -0.035       0.305
gdplr_bj_s1     0.9935      0.037     26.537      0.000       0.915       1.073
d_gdp1r_bj      6.1976      0.285     21.764      0.000       5.597       6.798
==============================================================================
Omnibus:                        2.799   Durbin-Watson:                   1.673
Prob(Omnibus):                  0.247   Jarque-Bera (JB):                1.433
Skew:                          -0.303   Prob(JB):                        0.488
Kurtosis:                       1.872   Cond. No.                     4.14e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.14e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [114]:
results.predict()

array([126.82820558, 129.03194867, 130.07270933, 133.38891309,
       137.72502094, 141.37904066, 142.38851322, 141.8252671 ,
       138.50468726, 138.98539273, 139.88213844, 142.42550053,
       150.03792225, 147.17252221, 148.18577519, 152.51597548,
       156.50725341, 158.46014552, 139.83986369, 128.98768591,
       119.96448788])

In [115]:
test1["gdp1r_bj"]

obs
1997    126.272872
1998    127.361097
1999    130.820954
2000    134.439627
2001    138.971821
2002    142.686523
2003    140.573841
2004    140.391340
2005    137.756210
2006    138.672350
2007    141.232242
2008    142.724052
2009    149.237610
2010    147.165622
2011    148.418572
2012    153.059582
2013    157.533902
2014    157.515500
2015    140.400000
2016    128.226331
2017    120.648922
Name: gdp1r_bj, dtype: float64

## Eq2: gdpn_bj 定义方程 ok
定义方程 ： gdpn_bj  = gdp1n_bj  + gdp2n_bj  + gdp3n_bj

In [14]:
whereis(["gdpn_bj","gdp1n_bj","gdp2n_bj","gdp3n_bj"])

gdpn_bj 在第  1  个数据框中
gdp1n_bj 在第  1  个数据框中
gdp2n_bj 在第  1  个数据框中
gdp3n_bj 在第  1  个数据框中


In [15]:
test1 = df_es[["gdpn_bj","gdp1n_bj","gdp2n_bj","gdp3n_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp1n_bj"] + test1["gdp2n_bj"] + test1["gdp3n_bj"]
test1.head(2)

,gdpn_bj,gdp1n_bj,gdp2n_bj,gdp3n_bj,验证结果
obs,,,,,
1997,2096.8,77.2,783.6,1236.0,2096.8
1998,2406.2,77.9,844.4,1483.9,2406.2


## Eq3 ：gdpr_bj 定义方程 ok
定义方程 ： gdpr_bj  = gdp1r_bj  + gdp2r_bj  + gdp3r_bj

In [16]:
whereis(["gdpr_bj","gdp1r_bj","gdp2r_bj","gdp3r_bj"])

gdpr_bj 在第  1  个数据框中
gdp1r_bj 在第  1  个数据框中
gdp2r_bj 在第  1  个数据框中
gdp3r_bj 在第  1  个数据框中


In [17]:
test1 = df_es[["gdpr_bj","gdp1r_bj","gdp2r_bj","gdp3r_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp1r_bj"] + test1["gdp2r_bj"] + test1["gdp3r_bj"]
test1.head(2)

,gdpr_bj,gdp1r_bj,gdp2r_bj,gdp3r_bj,验证结果
obs,,,,,
1997,3954.928483,126.272872,958.614649,2870.040962,3954.928483
1998,4334.706383,127.361097,1049.066999,3158.278288,4334.706383


## Eq4 : gdp1n_bj 定义方程 ok
定义方程 ： gdp1n_bj  = gdp1r_bj  * gdp1d_bj  / 100

In [18]:
whereis(["gdp1n_bj","gdp1r_bj","gdp1d_bj"])

gdp1n_bj 在第  1  个数据框中
gdp1r_bj 在第  1  个数据框中
gdp1d_bj 在第  1  个数据框中


In [19]:
test1 = df_es[["gdp1n_bj","gdp1r_bj","gdp1d_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp1r_bj"]*test1["gdp1d_bj"]/100
test1.head(2)

,gdp1n_bj,gdp1r_bj,gdp1d_bj,验证结果
obs,,,,
1997,77.2,126.272872,61.137439,77.2
1998,77.9,127.361097,61.164674,77.9


## Eq5 : gdp2_indan_bj 定义方程 ok
定义方程 ： gdp2_indan_bj  = gdp2_indar_bj  * gdp2_indad_bj  / 100


In [20]:
whereis(["gdp2_indan_bj","gdp2_indar_bj","gdp2_indad_bj"])

gdp2_indan_bj 在第  1  个数据框中
gdp2_indar_bj 在第  1  个数据框中
gdp2_indad_bj 在第  3  个数据框中


In [21]:
test1 = pd.concat([df_es[["gdp2_indan_bj","gdp2_indar_bj"]],df_der["gdp2_indad_bj"]],axis =1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp2_indar_bj"]*test1["gdp2_indad_bj"]/100
test1.head(2)

,gdp2_indan_bj,gdp2_indar_bj,gdp2_indad_bj,验证结果
obs,,,,
1997,639.7,752.168139,85.047474,639.7
1998,676.0,818.444916,82.595662,676.0


## Eq6 ：gdp2_bindan_bj 定义方程 ok
定义方程 ： gdp2_bindan_bj  = gdp2_bindar_bj  * gdp2_bindad_bj  / 100

In [22]:
whereis(["gdp2_bindan_bj","gdp2_bindar_bj","gdp2_bindad_bj"])

gdp2_bindan_bj 在第  1  个数据框中
gdp2_bindar_bj 在第  1  个数据框中
gdp2_bindad_bj 在第  3  个数据框中


In [23]:
test1 = pd.concat([df_es[["gdp2_bindan_bj","gdp2_bindar_bj"]],df_der["gdp2_bindad_bj"]],axis =1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp2_bindar_bj"]*test1["gdp2_bindad_bj"]/100
test1.head(2)

,gdp2_bindan_bj,gdp2_bindar_bj,gdp2_bindad_bj,验证结果
obs,,,,
1997,146.5,197.000053,74.365462,146.5
1998,171.0,225.188061,75.936530,171.0


## Eq7 : gdp2n_bj 定义方程 ok
定义方程 ： gdp2n_bj  = gdp2r_bj  * gdp2d_bj  / 100


In [24]:
whereis(["gdp2n_bj","gdp2r_bj","gdp2d_bj"])

gdp2n_bj 在第  1  个数据框中
gdp2r_bj 在第  1  个数据框中
gdp2d_bj 在第  1  个数据框中


In [25]:
test1 = df_es[["gdp2n_bj","gdp2r_bj","gdp2d_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp2r_bj"]*test1["gdp2d_bj"]/100
test1.head(2)

,gdp2n_bj,gdp2r_bj,gdp2d_bj,验证结果
obs,,,,
1997,783.6,958.614649,81.742961,783.6
1998,844.4,1049.066999,80.490569,844.4


## Eq8 ： gdp2r_bj2 定义方程 ok
定义方程 ： gdp2r_bj2  = gdp2_indar_bj  + gdp2_bindar_bj  + gdp2r_erro_bj

In [26]:
whereis(["gdp2r_bj2","gdp2_indar_bj","gdp2_bindar_bj","gdp2r_erro_bj"])

gdp2r_bj2 在第  1  个数据框中
gdp2_indar_bj 在第  1  个数据框中
gdp2_bindar_bj 在第  1  个数据框中
gdp2r_erro_bj 在第  3  个数据框中


In [27]:
test1 = pd.concat([df_es[["gdp2r_bj2","gdp2_indar_bj","gdp2_bindar_bj"]],df_der["gdp2r_erro_bj"]],axis =1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp2_indar_bj"] + test1["gdp2_bindar_bj"] + test1["gdp2r_erro_bj"]
test1.head(2)

,gdp2r_bj2,gdp2_indar_bj,gdp2_bindar_bj,gdp2r_erro_bj,验证结果
obs,,,,,
1997,958.614649,752.168139,197.000053,9.446457,958.614649
1998,1049.066999,818.444916,225.188061,5.434022,1049.066999


## Eq9 : gdp2r_bj1 行为方程 ok
行为方程 ： gdp2r_bj1  = @coef(1)  + @coef(2) * gdp_tecn_bj / gdp2d_bj  + @coef(3) * gdp2r_bj1(-1)  + @coef(4) * d_gdp2r_bj_1

- gdp_tecn_bj / gdp2d_bj : 可以将表达式写入方程中，但参数估计结果不一样

In [28]:
whereis(["gdp2r_bj1","gdp_tecn_bj","gdp2d_bj","d_gdp2r_bj_1"])

gdp2r_bj1 在第  1  个数据框中
gdp_tecn_bj 在第  1  个数据框中
gdp2d_bj 在第  1  个数据框中
d_gdp2r_bj_1 在第  2  个数据框中


In [29]:
test1 = pd.concat([df_es[["gdp2r_bj1","gdp_tecn_bj","gdp2d_bj"]],df_other["d_gdp2r_bj_1"],df_der["gdp2r_bj1_s1"]],axis=1)
test1["gdp_tecn_gdp2d"] = test1["gdp_tecn_bj"]/test1["gdp2d_bj"]
test1 = test1[(test1.index>2003) & (test1.index <2019)]
test1.head(2)

,gdp2r_bj1,gdp_tecn_bj,gdp2d_bj,d_gdp2r_bj_1,gdp2r_bj1_s1,gdp_tecn_gdp2d
obs,,,,,,
2004,2023.418698,370.6,92.304178,0.0,1725.142316,4.014986
2005,2224.489155,504.4,91.958192,-0.5,2023.418698,5.485101


In [30]:
#results = sm.ols(formula = "gdp2r_bj1 ~ gdp_tecn_bj/gdp2d_bj  + gdp2r_bj1.shift(1) + d_gdp2r_bj_1",data = test1 ).fit()
results = sm.ols(formula = "gdp2r_bj1 ~ gdp_tecn_gdp2d  + gdp2r_bj1_s1 + d_gdp2r_bj_1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              gdp2r_bj1   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.283e+04
Date:                Wed, 08 Jul 2020   Prob (F-statistic):           9.06e-20
Time:                        03:44:08   Log-Likelihood:                -64.871
No. Observations:                  15   AIC:                             137.7
Df Residuals:                      11   BIC:                             140.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        342.4635     33.177     10.322      0.000     269.442     415.485
gdp_tecn_gdp2d     1.4938      0.641      2.330      0.040       0.082       2.905
gdp2r_bj1_s1       0.9700      0.014     69.798      0.000       0.939       1.001
d_gdp2r_bj_1     155.2133      8.618     18.011      0.000     136.246     174.180
==============================================================================
Omnibus:                        1.380   Durbin-Watson:                   2.420
Prob(Omnibus):                  0.501   Jarque-Bera (JB):                1.054
Skew:                          -0.593   Prob(JB):                        0.590
Kurtosis:                       2.470   Cond. No.                     2.17e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq10 ：gdp2r_bj 定义方程  ok
定义方程 ： gdp2r_bj  = (gdp2r_bj1  + gdp2r_bj2)  / 2

In [31]:
whereis(["gdp2r_bj","gdp2r_bj1","gdp2r_bj2"])

gdp2r_bj 在第  1  个数据框中
gdp2r_bj1 在第  1  个数据框中
gdp2r_bj2 在第  1  个数据框中


In [32]:
test1 = df_es[["gdp2r_bj","gdp2r_bj1","gdp2r_bj2"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = (test1["gdp2r_bj1"] + test1["gdp2r_bj2"])/2
test1.head(2)

,gdp2r_bj,gdp2r_bj1,gdp2r_bj2,验证结果
obs,,,,
1997,958.614649,958.614649,958.614649,958.614649
1998,1049.066999,1049.066999,1049.066999,1049.066999


## Eq11 :gdp3n_bj 定义方程 ok
定义方程 ： gdp3n_bj  = gdp3r_bj  * gdp3d_bj  / 100

In [33]:
whereis(["gdp3n_bj","gdp3r_bj","gdp3d_bj"])

gdp3n_bj 在第  1  个数据框中
gdp3r_bj 在第  1  个数据框中
gdp3d_bj 在第  1  个数据框中


In [34]:
test1 = df_es[["gdp3n_bj","gdp3r_bj","gdp3d_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = (test1["gdp3r_bj"] * test1["gdp3d_bj"])/100
test1.head(2)

,gdp3n_bj,gdp3r_bj,gdp3d_bj,验证结果
obs,,,,
1997,1236.0,2870.040962,43.065587,1236.0
1998,1483.9,3158.278288,46.984460,1483.9


## Eq12 : gdp2_indar_bj 行为方程 ok
log(gdp2_indar_bj)  = @coef(1)  + @coef(2) * log(consr_bj)  + @coef(3) * log(investr_bj)  + @coef(4) * log(exusd_bj * exrate_cn)  + @coef(5) * d_gdp2_indar_bj

In [35]:
whereis(["gdp2_indar_bj","consr_bj","investr_bj","exusd_bj","exrate_cn","d_gdp2_indar_bj"])

gdp2_indar_bj 在第  1  个数据框中
consr_bj 在第  1  个数据框中
investr_bj 在第  1  个数据框中
exusd_bj 在第  1  个数据框中
exrate_cn 在第  2  个数据框中
d_gdp2_indar_bj 在第  2  个数据框中


In [36]:
test1 = pd.concat([df_es[["gdp2_indar_bj","consr_bj","investr_bj","exusd_bj"]],df_other[["exrate_cn","d_gdp2_indar_bj"]]],axis=1)
test1 = test1[(test1.index>1995) & (test1.index <2019)]
test1.head(2)

,gdp2_indar_bj,consr_bj,investr_bj,exusd_bj,exrate_cn,d_gdp2_indar_bj
obs,,,,,,
1996,691.916524,1582.300379,1148.028576,81.1975,831.42,0.0
1997,752.168139,1710.591485,1225.309912,96.1103,828.98,0.0


In [37]:
results = sm.ols(formula = "np.log(gdp2_indar_bj) ~ np.log(consr_bj)  + np.log(investr_bj) + np.log(exusd_bj * exrate_cn) + d_gdp2_indar_bj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(gdp2_indar_bj)   R-squared:                       0.997
Model:                               OLS   Adj. R-squared:                  0.996
Method:                    Least Squares   F-statistic:                     1529.
Date:                   Wed, 08 Jul 2020   Prob (F-statistic):           1.61e-22
Time:                           03:44:08   Log-Likelihood:                 46.404
No. Observations:                     23   AIC:                            -82.81
Df Residuals:                         18   BIC:                            -77.13
Df Model:                              4                                         
Covariance Type:               nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.1461      0.193     -0.757      0.459      -0.552       0.260
np.log(consr_bj)                 0.2604      0.092      2.840      0.011       0.068       0.453
np.log(investr_bj)               0.5564      0.104      5.365      0.000       0.339       0.774
np.log(exusd_bj * exrate_cn)     0.0796      0.037      2.159      0.045       0.002       0.157
d_gdp2_indar_bj                  0.0901      0.024      3.682      0.002       0.039       0.141
==============================================================================
Omnibus:                        0.116   Durbin-Watson:                   1.399
Prob(Omnibus):                  0.944   Jarque-Bera (JB):                0.067
Skew:                          -0.078   Prob(JB):                        0.967
Kurtosis:                       2.786   Cond. No.                         465.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq13 ：gdp2_bindar_bj 行为方程 NO 系数表达不一样

In [38]:
whereis(["gdp2_bindar_bj","investr_bj"])

gdp2_bindar_bj 在第  1  个数据框中
investr_bj 在第  1  个数据框中


In [116]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
test1 = df_es[["gdp2_bindar_bj","investr_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2019)]
ar = (0,1) 
ma = 0
mod = SARIMAX(endog = np.log(test1["gdp2_bindar_bj"])  , exog = np.log(test1["investr_bj"]) ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()

results.summary() #results.summary().tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:         gdp2_bindar_bj   No. Observations:                   22
Model:             SARIMAX([2], 0, 0)   Log Likelihood                  36.230
Date:                Wed, 08 Jul 2020   AIC                            -64.459
Time:                        10:13:11   BIC                            -60.476
Sample:                    01-01-1970   HQIC                           -63.682
                         - 01-01-1970                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0572      0.139     -0.411      0.681      -0.330       0.216
investr_bj     0.3882      0.149      2.604      0.009       0.096       0.680
ar.L2          1.0515      0.058     18.000      0.000       0.937       1.166
sigma2         0.0016      0.001      2.277      0.023       0.000       0.003
===================================================================================
Ljung-Box (Q):                       22.17   Jarque-Bera (JB):                 0.66
Prob(Q):                              0.28   Prob(JB):                         0.72
Heteroskedasticity (H):               0.41   Skew:                            -0.35
Prob(H) (two-sided):                  0.27   Kurtosis:                         2.44
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [118]:
predict = np.exp(results.predict(exog = np.log(df_es["investr_bj"])))
#re1 = pd.DataFrame(test1.index , predict)
re1

,obs
7,1998
244,1999
153,2000
273,2001
300,2002
329,2003
356,2004
378,2005
414,2006
475,2007


## Eq14 ：gdp3r_bj 行为方程  ok
行为方程 ：log(gdp3r_bj)  = @coef(1)  + @coef(2) * log(gdp2r_bj)  + @coef(3) * log(exusd_bj * exrate_cn)  + @coef(4) * log(gdp3r_bj(-1))

In [123]:
whereis(["gdp3r_bj","gdp2r_bj","exusd_bj","exrate_cn"])

gdp3r_bj 在第  1  个数据框中
gdp2r_bj 在第  1  个数据框中
exusd_bj 在第  1  个数据框中
exrate_cn 在第  2  个数据框中


In [132]:
test1 = pd.concat([df_es[["gdp3r_bj","gdp2r_bj","exusd_bj"]],df_other["exrate_cn"]],axis =1)
test1["exusexrte"] = test1["exusd_bj"]*test1["exrate_cn"]
test1["gdp3r_bj_s1"] = test1["gdp3r_bj"].shift(1)           
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,gdp3r_bj,gdp2r_bj,exusd_bj,exrate_cn,exusexrte,gdp3r_bj_s1
obs,,,,,,
1997,2870.040962,958.614649,96.1103,828.98,79673.516494,2567.476266
1998,3158.278288,1049.066999,105.1293,827.91,87037.598763,2870.040962


In [133]:
results = sm.ols(formula = "np.log(gdp3r_bj) ~ np.log(gdp2r_bj)  + np.log(exusexrte) +  np.log(gdp3r_bj_s1)",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(gdp3r_bj)   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.808e+04
Date:                Wed, 08 Jul 2020   Prob (F-statistic):           3.27e-33
Time:                        10:28:17   Log-Likelihood:                 70.938
No. Observations:                  22   AIC:                            -133.9
Df Residuals:                      18   BIC:                            -129.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0381      0.058      0.659      0.518      -0.083       0.159
np.log(gdp2r_bj)        0.2144      0.069      3.094      0.006       0.069       0.360
np.log(exusexrte)       0.0333      0.011      3.040      0.007       0.010       0.056
np.log(gdp3r_bj_s1)     0.7745      0.053     14.716      0.000       0.664       0.885
==============================================================================
Omnibus:                        0.367   Durbin-Watson:                   0.973
Prob(Omnibus):                  0.832   Jarque-Bera (JB):                0.481
Skew:                           0.249   Prob(JB):                        0.786
Kurtosis:                       2.474   Cond. No.                         691.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq15：con_fr_bj 定义方程 NO  结果不一致(都是数据源？）

定义方程 ： con_fr_bj  = con_pr_bj  + con_gr_bj


In [134]:
whereis(["con_fr_bj" ,"con_pr_bj" , "con_gr_bj"])

con_fr_bj 在第  1  个数据框中
con_pr_bj 在第  1  个数据框中
con_gr_bj 在第  1  个数据框中


In [138]:
test1 = df_es[["con_fr_bj" ,"con_pr_bj" , "con_gr_bj"]]
test1["验证结果"] = (test1["con_pr_bj"] + test1["con_gr_bj"])
test1.tail(4)

,con_fr_bj,con_pr_bj,con_gr_bj,验证结果
obs,,,,
2017,16366.882105,11137.377728,5228.473914,16365.851642
2018,17627.132027,11794.483014,5829.748414,17624.231428
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq16：con_gr_bj 定义方程 ok
定义方程 ： con_gr_bj  = con_gn_bj  / pcon_gd_bj  * 100

In [11]:
whereis(["con_gr_bj" ,"con_gn_bj" , "pcon_gd_bj"])

con_gr_bj 在第  1  个数据框中
con_gn_bj 在第  1  个数据框中


In [17]:
test1 = pd.concat([df_es[["con_gr_bj" ,"con_gn_bj"]],df_other["pcon_gd_bj"]],axis =1)
test1["验证结果"] = (test1["con_gn_bj"] /test1["pcon_gd_bj"])*100
test1.tail(4)

,con_gr_bj,con_gn_bj,pcon_gd_bj,验证结果
obs,,,,
2017,5228.473914,5350.60,103.179584,5185.715815
2018,5829.748414,6055.97,105.582669,5735.761409
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq17 ： con_pr_bj 定义方程 ok

定义方程 ： con_pr_bj  = con_pur_bj  + con_prr_bj


In [18]:
whereis(["con_pr_bj" ,"con_pur_bj" , "con_prr_bj"])

con_pr_bj 在第  1  个数据框中
con_pur_bj 在第  1  个数据框中
con_prr_bj 在第  1  个数据框中


In [20]:
test1 = df_es[["con_pr_bj" ,"con_pur_bj" , "con_prr_bj"]]
test1["预测"] = (test1["con_pur_bj"] + test1["con_prr_bj"])
test1.tail(4)

,con_pr_bj,con_pur_bj,con_prr_bj,预测
obs,,,,
2017,11137.377728,10390.265568,747.112160,11137.377728
2018,11794.483014,10992.900971,801.582043,11794.483014
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq18 ：con_fn_bj 定义方程 ok
con_fn_bj  = con_pn_bj  + con_gn_bj

In [21]:
whereis(["con_fn_bj" ,"con_pn_bj" , "con_gn_bj"])

con_fn_bj 在第  1  个数据框中
con_pn_bj 在第  1  个数据框中
con_gn_bj 在第  1  个数据框中


In [22]:
test1 = df_es[["con_fn_bj" ,"con_pn_bj" , "con_gn_bj"]]
test1["预测"] = (test1["con_pn_bj"] + test1["con_gn_bj"])
test1.tail(4)

,con_fn_bj,con_pn_bj,con_gn_bj,预测
obs,,,,
2017,16842.1,11491.50,5350.60,16842.1
2018,18508.9,12452.93,6055.97,18508.9
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq19 ：con_pn_bj 定义方程 ok
con_pn_bj  = con_pun_bj  + con_prn_bj

In [23]:
whereis(["con_pn_bj" ,"con_pun_bj" , "con_prn_bj"])

con_pn_bj 在第  1  个数据框中
con_pun_bj 在第  1  个数据框中
con_prn_bj 在第  1  个数据框中


In [25]:
test1 = df_es[["con_pn_bj" ,"con_pun_bj" , "con_prn_bj"]]
test1["预测"] = (test1["con_pun_bj"] + test1["con_prn_bj"])
test1.tail(4)

,con_pn_bj,con_pun_bj,con_prn_bj,预测
obs,,,,
2017,11491.50,10724.000000,767.500000,11491.50
2018,12452.93,11615.927984,837.002016,12452.93
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq20 ：con_pun_bj 定义方程 ok
con_pun_bj  = con_pur_bj  * pcon_pud_bj  / 100

In [30]:
whereis(["con_pun_bj" ,"con_pur_bj" , "pcon_pud_bj"])

con_pun_bj 在第  1  个数据框中
con_pur_bj 在第  1  个数据框中


In [32]:
test1 = pd.concat([df_es[["con_pun_bj" ,"con_pur_bj"]],df_other["pcon_pud_bj"]],axis =1)
test1["验证结果"] = (test1["con_pur_bj"] * test1["pcon_pud_bj"])/100
test1.tail(4)

,con_pun_bj,con_pur_bj,pcon_pud_bj,验证结果
obs,,,,
2017,10724.000000,10390.265568,103.211991,10724.000005
2018,11615.927984,10992.900971,105.667540,11615.927987
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq21 ：con_prn_bj 定义方程 ok
con_prn_bj  = con_prr_bj  * pcon_prd_bj  / 100

In [35]:
whereis(["con_prn_bj" ,"con_prr_bj" , "pcon_prd_bj"])

con_prn_bj 在第  1  个数据框中
con_prr_bj 在第  1  个数据框中


In [36]:
test1 = pd.concat([df_es[["con_prn_bj" ,"con_prr_bj"]],df_other["pcon_prd_bj"]],axis =1)
test1["验证结果"] = (test1["con_prr_bj"] * test1["pcon_prd_bj"])/100
test1.tail(4)

,con_prn_bj,con_prr_bj,pcon_prd_bj,验证结果
obs,,,,
2017,767.500000,747.112160,102.728886,767.500000
2018,837.002016,801.582043,104.418758,837.002016
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq22 : con_gn_bj 行为方程 ok
con_gn_bj  = @coef(1)  + @coef(2) * fen_bj

In [37]:
whereis(["con_gn_bj" ,"fen_bj" ])

con_gn_bj 在第  1  个数据框中
fen_bj 在第  1  个数据框中


In [38]:
test1 = df_es[["con_gn_bj","fen_bj"]]
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,con_gn_bj,fen_bj
obs,,
1997,205.4,236.39
1998,238.4,280.68


In [39]:
results = sm.ols(formula = "con_gn_bj ~ fen_bj",data = test1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              con_gn_bj   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     2370.
Date:                Thu, 09 Jul 2020   Prob (F-statistic):           2.97e-22
Time:                        02:07:18   Log-Likelihood:                -143.60
No. Observations:                  22   AIC:                             291.2
Df Residuals:                      20   BIC:                             293.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    100.0392     56.115      1.783      0.090     -17.014     217.092
fen_bj         0.7902      0.016     48.686      0.000       0.756       0.824
==============================================================================
Omnibus:                        1.814   Durbin-Watson:                   1.059
Prob(Omnibus):                  0.404   Jarque-Bera (JB):                0.560
Skew:                           0.244   Prob(JB):                        0.756
Kurtosis:                       3.610   Cond. No.                     5.25e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.25e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq23 ： investn_bj 定义方程 ok
investn_bj  = investr_bj  * faipi_bj  / 100

In [40]:
whereis(["investn_bj" ,"investr_bj" , "faipi_bj"])

investn_bj 在第  1  个数据框中
investr_bj 在第  1  个数据框中
faipi_bj 在第  1  个数据框中


In [41]:
test1 = df_es[["investn_bj" ,"investr_bj" , "faipi_bj"]]
test1["预测"] = (test1["investr_bj"] * test1["faipi_bj"]) /100
test1.tail(4)

,investn_bj,investr_bj,faipi_bj,预测
obs,,,,
2017,8948.1000,8572.134742,104.385900,8948.1000
2018,8062.2381,7440.745089,108.352564,8062.2381
2019,NaN,NaN,110.627968,NaN
2020,NaN,NaN,NaN,NaN


## Eq24 : inv_fon_bj 定义方程  ok
inv_fon_bj  = inv_for_bj  * pinv_fod_bj  / 100

In [61]:
whereis(["inv_fon_bj" ,"inv_for_bj" , "pinv_fod_bj"])

inv_fon_bj 在第  1  个数据框中
inv_fon_bj 在第  2  个数据框中
inv_for_bj 在第  1  个数据框中


In [62]:
df_other.columns

Index(['d_gdp1r_bj', 'cul_area_bj', 'd_gdp2r_bj_1', 'exrate_cn',
       'd_gdp2_indar_bj', 'pcon_gd_bj', 'pcon_pud_bj', 'pcon_prd_bj',
       'pinv_fod_bj'],
      dtype='object')

In [63]:
test1 = pd.concat([df_es[["inv_fon_bj" ,"inv_for_bj"]],df_other["pinv_fod_bj"]],axis =1)
test1["验证结果"] = (test1["inv_for_bj"] * test1["pinv_fod_bj"])/100
test1.tail(4)

,inv_fon_bj,inv_for_bj,pinv_fod_bj,验证结果
obs,,,,
2017,10946.30,10492.500014,104.324994,10946.300000
2018,11286.03,10702.350015,105.453755,11286.029996
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq25 : inv_chn_bj 定义方程 ok
inv_chn_bj  = inv_chr_bj  * pinv_chd_bj  / 100

In [15]:
whereis(["inv_chn_bj" ,"inv_chr_bj" , "pinv_chd_bj"])

inv_chn_bj 在第  1  个数据框中
inv_chr_bj 在第  2  个数据框中
pinv_chd_bj 在第  2  个数据框中


In [17]:
test1 = pd.concat([df_es["inv_chn_bj" ],df_other[["inv_chr_bj" , "pinv_chd_bj"]]],axis =1)
test1["预测"] = (test1["inv_chr_bj"] * test1["pinv_chd_bj"])/100
test1.head(2)

,inv_chn_bj,inv_chr_bj,pinv_chd_bj,预测
obs,,,,
1996,85.2,NaN,NaN,NaN
1997,195.1,NaN,NaN,NaN


## Eq 26 : consr_bj   定义方程  no 预测与真实值相差较大
consr_bj  = consn_bj  / rpi_bj  * 100

In [33]:
whereis(["consr_bj" ,"consn_bj" , "rpi_bj"])

consr_bj 在第  1  个数据框中
consn_bj 在第  1  个数据框中
consn_bj 在第  2  个数据框中
rpi_bj 在第  1  个数据框中


In [36]:
test1 = df_es[["consr_bj" ,"consn_bj" , "rpi_bj"]]
test1["预测"] = (test1["consn_bj"] / test1["rpi_bj"])*100
test1.head(2)

,consr_bj,consn_bj,rpi_bj,预测
obs,,,,
1996,1582.300379,1061.6,101.439268,1046.537519
1997,1710.591485,1208.5,105.293960,1147.739146


## Eq 27 :  inv_for_bj  定义方程 ok
inv_for_bj  = inv_fir_bj  + inv_chr_bj

In [35]:
whereis(["inv_for_bj" ,"inv_fir_bj" , "inv_chr_bj"])

inv_for_bj 在第  1  个数据框中
inv_fir_bj 在第  1  个数据框中
inv_fir_bj 在第  2  个数据框中
inv_chr_bj 在第  2  个数据框中


In [43]:
test1 = pd.concat([df_es[["inv_for_bj" ,"inv_fir_bj"]] , df_other["inv_chr_bj"]],axis =1)
test1["预测"] = test1["inv_fir_bj"] + test1["inv_chr_bj"]
test1.tail(4)

,inv_for_bj,inv_fir_bj,inv_chr_bj,预测
obs,,,,
2017,10492.500014,9909.109291,583.390724,10492.500014
2018,10702.350015,10156.837023,545.512992,10702.350015
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq 28 : inv_fin_bj   定义方程 ok
inv_fin_bj  = inv_fir_bj  * pinv_fid_bj  / 100

In [45]:
whereis(["inv_fin_bj" ,"inv_fir_bj" , "pinv_fid_bj"])

inv_fin_bj 在第  1  个数据框中
inv_fir_bj 在第  1  个数据框中
inv_fir_bj 在第  2  个数据框中
pinv_fid_bj 在第  2  个数据框中


In [48]:
test1 = pd.concat([df_es[["inv_fin_bj" ,"inv_fir_bj"]] , df_other["pinv_fid_bj"]],axis =1)
test1["预测"] = (test1["inv_fir_bj"] * test1["pinv_fid_bj"])/100
test1.tail(4)

,inv_fin_bj,inv_fir_bj,pinv_fid_bj,预测
obs,,,,
2017,10375.30,9909.109291,104.704668,10375.300004
2018,10801.15,10156.837023,106.343638,10801.149996
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq 29 :  gdper_bj  定义方程 ok
gdper_bj  = con_fr_bj  + inv_for_bj  + nex_gsr_bj

In [46]:
whereis(["gdper_bj" ,"con_fr_bj" , "inv_for_bj","nex_gsr_bj"])

gdper_bj 在第  1  个数据框中
con_fr_bj 在第  1  个数据框中
con_fr_bj 在第  2  个数据框中
inv_for_bj 在第  1  个数据框中
nex_gsr_bj 在第  1  个数据框中
nex_gsr_bj 在第  2  个数据框中


In [49]:
test1 = df_es[["gdper_bj" ,"con_fr_bj","inv_for_bj","nex_gsr_bj"]]
test1["预测"] = test1["con_fr_bj"] + test1["inv_for_bj"] + test1["nex_gsr_bj"]
test1.tail(4)

,gdper_bj,con_fr_bj,inv_for_bj,nex_gsr_bj,预测
obs,,,,,
2017,26991.077315,16366.882105,10492.500014,131.695195,26991.077315
2018,28772.759500,17627.132027,10702.350015,443.277458,28772.759500
2019,30527.897830,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN


## Eq 30 : gdpen_bj   定义方程 ok
gdpen_bj  = con_fn_bj  + inv_fon_bj  + nex_gsn_bj

In [51]:
whereis(["gdpen_bj" ,"con_fn_bj" , "inv_fon_bj","nex_gsn_bj"])

gdpen_bj 在第  1  个数据框中
con_fn_bj 在第  1  个数据框中
con_fn_bj 在第  2  个数据框中
inv_fon_bj 在第  1  个数据框中
nex_gsn_bj 在第  1  个数据框中


In [52]:
test1 = df_es[["gdpen_bj" ,"con_fn_bj" , "inv_fon_bj","nex_gsn_bj"]]
test1["预测"] = test1["con_fn_bj"] + test1["inv_fon_bj"] + test1["nex_gsn_bj"]
test1.tail(4)

,gdpen_bj,con_fn_bj,inv_fon_bj,nex_gsn_bj,预测
obs,,,,,
2017,28014.9,16842.1,10946.30,226.5,28014.90
2018,30320.0,18508.9,11286.03,525.1,30320.03
2019,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN


## Eq 31 :  nex_gsn_bj   定义方程 ok
nex_gsn_bj  = nex_gsr_bj  * pnex_gsd_bj  / 100


In [53]:
whereis(["nex_gsn_bj" ,"nex_gsr_bj" , "pnex_gsd_bj"])

nex_gsn_bj 在第  1  个数据框中
nex_gsr_bj 在第  1  个数据框中
nex_gsr_bj 在第  2  个数据框中
pnex_gsd_bj 在第  2  个数据框中


In [54]:
test1 = pd.concat([df_es[["nex_gsn_bj" ,"nex_gsr_bj"]] , df_other["pnex_gsd_bj"]],axis =1)
test1["预测"] = (test1["nex_gsr_bj"] * test1["pnex_gsd_bj"])/100
test1.tail(4)

,nex_gsn_bj,nex_gsr_bj,pnex_gsd_bj,预测
obs,,,,
2017,226.5,131.695195,171.988051,226.5
2018,525.1,443.277458,118.458539,525.1
2019,NaN,NaN,81.589537,NaN
2020,NaN,NaN,NaN,NaN


## Eq32 : con_pur_bj 行为方程 ok
con_pur_bj  = @coef(1)  + @coef(2) * yht_un_bj / pcon_pud_bj * 100

In [55]:
whereis(["con_pur_bj" ,"yht_un_bj" , "pcon_pud_bj"])

con_pur_bj 在第  1  个数据框中
yht_un_bj 在第  1  个数据框中
pcon_pud_bj 在第  2  个数据框中


In [60]:
test1 = pd.concat([df_es[["con_pur_bj" ,"yht_un_bj"]],df_other["pcon_pud_bj"]],axis=1)
test1["yht_pcon_100"] = test1["yht_un_bj"]/test1["pcon_pud_bj"]*100
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,con_pur_bj,yht_un_bj,pcon_pud_bj,yht_pcon_100
obs,,,,
2006,3705.167054,2891.476489,72.975723,3962.244413
2007,4079.388927,3363.789827,79.804149,4215.056310


In [61]:
results = sm.ols(formula = "con_pur_bj ~ yht_pcon_100",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             con_pur_bj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.063e+04
Date:                Thu, 09 Jul 2020   Prob (F-statistic):           8.94e-18
Time:                        07:22:42   Log-Likelihood:                -74.748
No. Observations:                  13   AIC:                             153.5
Df Residuals:                      11   BIC:                             154.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      160.4162     72.731      2.206      0.050       0.336     320.497
yht_pcon_100     0.9123      0.009    103.096      0.000       0.893       0.932
==============================================================================
Omnibus:                        5.568   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                1.379
Skew:                          -0.041   Prob(JB):                        0.502
Kurtosis:                       1.407   Cond. No.                     2.61e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.61e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq33 : con_prr_bj 行为方程 ok
eviews :function :d(x) :first difference ,d(x) = x - x(-1)

方程 ： con_prr_bj  = @coef(1)  + @coef(2) * yht_rn_bj / pcon_prd_bj * 100  + @coef(3) * d(con_prr_bj(-1))

In [62]:
whereis(["con_prr_bj" ,"yht_rn_bj" , "pcon_prd_bj"])

con_prr_bj 在第  1  个数据框中
yht_rn_bj 在第  1  个数据框中
pcon_prd_bj 在第  2  个数据框中


In [63]:
test1 = pd.concat([df_es[["con_prr_bj" ,"yht_rn_bj"]],df_other["pcon_prd_bj"]],axis=1)
test1["yht_pcon_100"] = test1["yht_rn_bj"]/test1["pcon_prd_bj"]*100
test1["diff_con_prr"] = test1["con_prr_bj"].shift(1)-test1["con_prr_bj"].shift(2)
test1 = test1[(test1.index>2007) & (test1.index <2019)]
test1.head(2)

,con_prr_bj,yht_rn_bj,pcon_prd_bj,yht_pcon_100,diff_con_prr
obs,,,,,
2008,346.029075,268.777750,89.695232,299.656676,36.040065
2009,384.730111,305.178093,94.787089,321.961669,32.030453


In [64]:
results = sm.ols(formula = "con_prr_bj ~ yht_pcon_100 + diff_con_prr",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             con_prr_bj   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     199.3
Date:                Thu, 09 Jul 2020   Prob (F-statistic):           1.50e-07
Time:                        07:36:35   Log-Likelihood:                -49.318
No. Observations:                  11   AIC:                             104.6
Df Residuals:                       8   BIC:                             105.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       65.9612     25.621      2.574      0.033       6.878     125.044
yht_pcon_100     0.9533      0.051     18.630      0.000       0.835       1.071
diff_con_prr     0.5949      0.263      2.263      0.053      -0.011       1.201
==============================================================================
Omnibus:                        0.898   Durbin-Watson:                   1.690
Prob(Omnibus):                  0.638   Jarque-Bera (JB):                0.603
Skew:                           0.511   Prob(JB):                        0.740
Kurtosis:                       2.480   Cond. No.                     1.72e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.72e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq34 : investr_bj 行为方程
investr_bj  = @coef(1)  + @coef(2) * loann_bj / faipi_bj * 100  + @coef(3) * d_investr_bj

In [66]:
whereis(["investr_bj" ,"loann_bj" , "faipi_bj","d_investr_bj"])

investr_bj 在第  1  个数据框中
loann_bj 在第  1  个数据框中
faipi_bj 在第  1  个数据框中


## Eq35 : inv_fir_bj 行为方程
inv_fir_bj  = @coef(1)  + @coef(2) * investr_bj  + @coef(3) * inv_fir_bj(-1)

In [67]:
whereis(["inv_fir_bj" ,"investr_bj" ])

inv_fir_bj 在第  1  个数据框中
inv_fir_bj 在第  2  个数据框中
investr_bj 在第  1  个数据框中


## Eq36 : nex_gsr_bj 行为方程
nex_gsr_bj  = @coef(1)  + @coef(2) * gdpr_bj  + @coef(3) * nex_gsr_bj(-1)  + @coef(4) * d_nex_gsr_bj

In [68]:
whereis(["nex_gsr_bj" ,"gdpr_bj","d_nex_gsr_bj" ])

nex_gsr_bj 在第  1  个数据框中
nex_gsr_bj 在第  2  个数据框中
gdpr_bj 在第  1  个数据框中


## Eq37 : consn_bj 行为方程
consn_bj  = @coef(1)  + @coef(2) * (yht_n_bj - yht_n_bj(-1))  + @coef(3) * consn_bj(-1)  + @coef(4) * d_consn_bj

In [69]:
whereis(["consn_bj" ,"yht_n_bj" , "d_consn_bj"])

consn_bj 在第  1  个数据框中
consn_bj 在第  2  个数据框中
yht_n_bj 在第  1  个数据框中


## Eq38: exusd_bj 行为方程
exusd_bj  = @coef(1)  + @coef(2) * exusd_ch  + @coef(3) * exusd_bj(-1)  + @coef(4) * (exusd_bj(-1) - exusd_bj(-2))  + @coef(5) * d_exusd_bj


In [74]:
whereis(["exusd_bj" ,"exusd_ch" , "d_exusd_bj"])

exusd_bj 在第  1  个数据框中


## Eq39 : imusd_bj 行为方程
log(imusd_bj)  = @coef(1)  + @coef(2) * log(imusd_bj(-1))  + @coef(3) * log(exusd_bj)  + @coef(4) * d_imusd_bj

In [73]:
whereis(["imusd_bj" ,"exusd_bj" , "d_imusd_bj"])

imusd_bj 在第  1  个数据框中
exusd_bj 在第  1  个数据框中


# 模型（联立方程）